In [ ]:
!pip install numpy pandas humanize

load data

In [47]:
import json
import os

import pandas as pd

# Get all files in the working directory that match the pattern "stats_*"
files = [f for f in os.listdir() if f.startswith('stats_') and f.endswith('.json')]

# Initialize an empty list to store the data
data = []
ids = []

# Loop through each file and load the JSON data
for file in files:
    with open(file) as f:
        file_data = json.load(f)
        # Add a column for the file id
        ids.append('.'.join(file.split('_')[1].split('.')[:-1]))

        # add only 'size' fields
        file_data = {
            k: v.get('size') if isinstance(v, dict) else v for k, v in file_data.items()
        }

        data.append(file_data)

analyse

In [74]:
import humanize

# Create a dataframe from the list of dictionaries
df = pd.DataFrame(data)
df['total_size'] = df.sum(axis=1)

# Add the ids as a column
df['id'] = ids
df = df.sort_values(by='id')

# add trend for all columns
for column in df.columns:
    if column not in ['id', 'total_size']:
        df[column + ' %'] = (
            df[column].pct_change().fillna(0).apply(lambda x: round(x * 100, 2))
        )

# order columns by name (excluding id and total_size)
df = df.reindex(
    ['id', 'total_size']
    + sorted([col for col in df.columns if col not in ['id', 'total_size']]),
    axis=1,
)

# add trends compared to the previous release as number an as percentage
df['change'] = df['total_size'].diff().fillna(0)
df['change %'] = round(df['total_size'].pct_change().fillna(0) * 100, 2)

df

,id,total_size,CSS,CSS %,Dockerfile,Dockerfile %,HTML,HTML %,JavaScript,JavaScript %,Procfile,Procfile %,Python,Python %,Shell,Shell %,TypeScript,TypeScript %,change,change %
3,0.15.8,7847805,247645,0.00,5734,0.00,651766,0.00,2119419,0.00,426,0.0,3784360,0.00,31782,0.00,1006673,0.00,0.0,0.00
0,0.16.9,8263250,247985,0.14,5777,0.75,656261,0.69,2138364,0.89,426,0.0,3993921,5.54,37297,17.35,1183219,17.54,415445.0,5.29
1,0.17.0,8901505,247894,-0.04,6139,6.27,659185,0.45,2141256,0.14,426,0.0,4258061,6.61,37305,0.02,1551239,31.10,638255.0,7.72
2,0.18.0,5767700,17146,-93.08,6139,0.00,54892,-91.67,5474,-99.74,426,0.0,4096824,-3.79,37305,0.00,1549494,-0.11,-3133805.0,-35.21


In [76]:
# display all nummeric columns in human format
df['total_size'] = df['total_size'].apply(lambda x: humanize.intword(x))
df['change'] = df['change'].apply(lambda x: humanize.intword(x))

df

,id,total_size,CSS,CSS %,Dockerfile,Dockerfile %,HTML,HTML %,JavaScript,JavaScript %,Procfile,Procfile %,Python,Python %,Shell,Shell %,TypeScript,TypeScript %,change,change %
3,0.15.8,7.8 million,247645,0.00,5734,0.00,651766,0.00,2119419,0.00,426,0.0,3784360,0.00,31782,0.00,1006673,0.00,0,0.00
0,0.16.9,8.3 million,247985,0.14,5777,0.75,656261,0.69,2138364,0.89,426,0.0,3993921,5.54,37297,17.35,1183219,17.54,415.4 thousand,5.29
1,0.17.0,8.9 million,247894,-0.04,6139,6.27,659185,0.45,2141256,0.14,426,0.0,4258061,6.61,37305,0.02,1551239,31.10,638.3 thousand,7.72
2,0.18.0,5.8 million,17146,-93.08,6139,0.00,54892,-91.67,5474,-99.74,426,0.0,4096824,-3.79,37305,0.00,1549494,-0.11,-3.1 million,-35.21
